<a href="https://colab.research.google.com/github/leearzuaga/Projects/blob/main/EbayCarDataProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Used Car Listings on eBay Kleinanzeigen

##### In this project, I'll be working with a dataset of used cars from *eBay Kleinanzeigen* which is a classifieds section of the German eBay website. The goal for this project is to clean the dataset so that it is easier to analyze and extract insights. 

##### Please note that the dataset I will be working with is not the original dataset scraped and uploaded [here]('https://data.world/data-society/used-cars-data'). Instead, the dataset I will be working with has been modified by dataquest to inlcude 50,000 data points from the original dataset for efficiency reasons.

##### Before I get started, I want to provide the data dictionary for the dataset I will be using so everyone can follow along. 

##### Data Dictionary:

column|description
--- | ---
dateCrawled | When this ad was first crawled. All field-values are taken from this date.
name | Name of the car.
seller | Whether the seller is private or a dealer.
offerType | The type of listing
price | The price on the ad to sell the car.
abtest | Whether the listing is included in an A/B test.
vehicleType | The vehicle Type.
yearOfRegistration | The year in which the car was first registered.
gearbox | The transmission type.
powerPS | The power of the car in PS.
model | The car model name.
kilometer | How many kilometers the car has driven.
monthOfRegistration | The month in which the car was first registered.
fuelType | What type of fuel the car uses.
brand | The brand of the car.
notRepairedDamage | If the car has a damage which is not yet repaired.
dateCreated | The date on which the eBay listing was created.
nrOfPictures | The number of pictures in the ad.
postalCode | The postal code for the location of the vehicle.
lastSeenOnline | When the crawler saw this ad last online.

#Importing Libraries And Reading In Data

In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')  # the defualt ETF-8 encoding settings did not work so I used the next most popular encoding

In [2]:
# The below code will give us information about our dataset
# and it will allow us to see the first 5 rows

autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


##### After examining the output above, there are a few things we should note. 

*   We have 50,000 rows and 20 columns
*   All but 5 of the 20 columns are non-object or string data types
*   Column names are in Camel case (offerType) format instead of Python's preferred Snake case format (offer_type).
*   5 out of the 20 columns contain null values

##### Now that we've made these observations, let us start cleaning the data and exploring it further.



#Data Cleaning

### Cleaning Column Names

##### I noticed in the previous section that the columns in our dataset our in a camel case format. In this section, I am going to clean up some of the column names by converting them from camel case to snake case. I'll also rename a few columns with more descriptive column names.

In [3]:
# First lets take a look at our column names one more time

autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
# In the above output we once again see the camel case colum names
# I'll be renaming these columns: yearOfRegistration, monthOfRegistration, notRepairedDamage, and dateCreated
# The code below allows me to rename columns and convert them to snake case all at once

autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gear_box', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuelt_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_of_photos', 'postal_code',
       'last_seen']


In [5]:
# Verifying the changes worked

print(autos.columns)
autos.head(3)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gear_box', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuelt_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_of_photos', 'postal_code',
       'last_seen'],
      dtype='object')


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_ps,model,odometer,registration_month,fuelt_type,brand,unrepaired_damage,ad_created,num_of_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


##### We can see from the above output that our changes were successful. These changes should allow to more easily navigate our dataset.

# More Exploration And Cleaning

##### Now that I've cleaned up some of the column names and converted them to snake case format, I will look for text columns where all or almost all values are the same (these columns can often be dropped as they don't provide useful information). I'll also look for numeric data stored as text which can be cleaned and converted to its proper data type (int or float). 

In [6]:
# The below code allows me to look at descriptive statistics for all the columns in our dataset
# We can use the unique descriptive statistic to identify any columns with all or almost all of the same values 

autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_ps,model,odometer,registration_month,fuelt_type,brand,unrepaired_damage,ad_created,num_of_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


##### After looking at the output above, here are some observations:


*   There are number of text columns with 2 or fewer values
*   The 'num_of_photos' column is numeric but its descriptive statistics are all zero. This column needs further investigating
*   The 'price' and 'odometer' columns are numeric values stored as text



In [7]:
# Of the number of text columns with 2 or fewer values, 'seller' and 'offer_type' are two 
# that wont further our analysis 

print(autos['seller'].value_counts())
print()
print(autos['seller'].head())
print()
print(autos['offer_type'].value_counts())
print()
print(autos['offer_type'].head())

privat        49999
gewerblich        1
Name: seller, dtype: int64

0    privat
1    privat
2    privat
3    privat
4    privat
Name: seller, dtype: object

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

0    Angebot
1    Angebot
2    Angebot
3    Angebot
4    Angebot
Name: offer_type, dtype: object


In [8]:
# After investigating the 'num_of_photos' column further, we can see from the output that the column is filled with zeros.

print(autos['num_of_photos'].value_counts())
print()
print(autos['num_of_photos'])

0    50000
Name: num_of_photos, dtype: int64

0        0
1        0
2        0
3        0
4        0
        ..
49995    0
49996    0
49997    0
49998    0
49999    0
Name: num_of_photos, Length: 50000, dtype: int64


In [9]:
# Because the 3 columns referenced above aren't going to further our analysis, I will drop them from the dataset

autos = autos.drop(["num_of_photos", "seller", "offer_type"], axis=1)

In [10]:
# Checking our column names again to confirm the useless ones above were dropped

autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gear_box', 'power_ps', 'model', 'odometer',
       'registration_month', 'fuelt_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

In [11]:
# Now we'll convert the price and odometer columns to their proper numeric data type
# To accomplish this task I will first remove any non-numeric characters (',', 'km', etc)
# Then I will convert the text values to their proper numeric data type

# Here is how each column looks like before conversion:

print(autos['odometer'].head(3))
print()
print(autos['price'].head(3))

0    150,000km
1    150,000km
2     70,000km
Name: odometer, dtype: object

0    $5,000
1    $8,500
2    $8,990
Name: price, dtype: object


In [12]:
# Notice all of the non-numeric characters in the above output

# The code below replaces the non-numeric characters and converts both columns to their proper data types

autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '').astype(int)
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(int)

# Here is how they columns look after converison

print(autos['odometer'].head(3))
print()
print(autos['price'].head(3))


0    150000
1    150000
2     70000
Name: odometer, dtype: int64

0    5000
1    8500
2    8990
Name: price, dtype: int64


In [13]:
# Since we removed the 'km' from the odometer column values, I will add it back to the column name to be more descriptive 

autos.rename({'odometer': 'odometer_km'}, axis = 1, inplace = True)

autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gear_box', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuelt_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

# Exploring the Odometer and Price Columns Further

### Checking For Outliers



In [14]:
print(autos['odometer_km'].shape)  # returns the number of values in this column
print()
print(autos['odometer_km'].describe())  # returns descriptive statistics of the values in the column
print()
print(autos['odometer_km'].value_counts().sort_index(ascending = False))  # returns count of unique values

(50000,)

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64


##### Looking at the above output I notice a few things:

* We have 50000 values in the 'odometer_km' column which means we don't have any missing values

* The odometers seem to be rounded but not sure why

* The vast majority of used cars in our dataset are high mileage vehicles

* Outside of those observations, the 'odometer_km' column doesn't seem to have any outliers

In [15]:
print(autos['price'].shape)
print()
print(autos['price'].describe())
print()
print(autos['price'].value_counts().sort_index(ascending = False).head(20))
print()
print(autos['price'].value_counts().sort_index(ascending = False).tail(20))

(50000,)

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

35       1
30       7
29       1
25       5
20       4
18       1
17       3
15       2
14       1
13       2
12       3
11       2
10       7
9        1
8        1
5        2
3        1
2        3
1      156
0     1421
Name: price, dtype: int64


##### After looking at the output above, I noticed a number of things:

* For an auction site, there seems to be a lot of cars sold for nothing
* Prices also seem to be rounded, again not sure why but could be users prefer to deal with round numbers?
* There is a huge jump in price after 350,000

##### Unlike the 'odometer_km' column, the 'price' column has a fair bit of outliers. Aside from the 1421 cars priced at 0, there's a fair bit of outliers showing for cars that have been sold above 350,000. Because prices seem to nonesensically spike after 350,000, we're going to consider those values outliers and remove them from our analysis.

In [16]:
# The code below will filter out the outliers from the 'price' column

autos[autos['price'].between(1, 351000)].describe()

,price,registration_year,power_ps,odometer_km,registration_month,postal_code
count,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000
mean,5888.935591,2004.755421,117.197158,125770.101925,5.782251,50975.745207
std,9059.854754,88.643887,200.649618,39788.636804,3.685595,25746.968398
min,1.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000,30657.000000
50%,3000.000000,2004.000000,107.000000,150000.000000,6.000000,49716.000000
75%,7490.000000,2008.000000,150.000000,150000.000000,9.000000,71665.000000
max,350000.000000,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


##### Notice how the count has changed from 50,000 to 48,565. Also notice the mean, median (50%), min, and max statistics have all changed.

# Exploring the Date Columns

##### There are 5 columns in our dataset that represent date values. Some of these values were created by the individual who created the dataset and some values were created by the website being scraped. We can refer to the data dictionary to differentiate these values:

  - `date_crawled` : added by the crawler
  - `last_seen` : added by the crawler
  - `ad_created` : from the website
  - `registration_month` : from the website
  - `registration_year` : from the website

##### The 'date_crawled', 'last_seen', and 'ad_created' columns are all identified as string values by pandas. As a result, we need to convert the data into a numerical representation so we can understand it quantitatively. Lets get started.

In [21]:
# First lets take a quick look at our 3 columns of interest to see 
# how the values in each column are formatted

autos[['date_crawled', 'last_seen', 'ad_created']][:5]

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


###### Notice from the output that the first 10 characters represent the day (e.g. 2016-03-26). To better understand the date range, I will extract just the date values to generate a distribution where I'll sort by index and values.

In [18]:
# The code below extracts the first 10 characters from the date time values

print(autos['date_crawled'].str[:10])
print()
print(autos['last_seen'].str[:10])
print()
print(autos['ad_created'].str[:10])

0        2016-03-26
1        2016-04-04
2        2016-03-26
3        2016-03-12
4        2016-04-01
            ...    
49995    2016-03-27
49996    2016-03-28
49997    2016-04-02
49998    2016-03-08
49999    2016-03-14
Name: date_crawled, Length: 50000, dtype: object

0        2016-04-06
1        2016-04-06
2        2016-04-06
3        2016-03-15
4        2016-04-01
            ...    
49995    2016-04-01
49996    2016-04-02
49997    2016-04-04
49998    2016-04-05
49999    2016-04-06
Name: last_seen, Length: 50000, dtype: object

0        2016-03-26
1        2016-04-04
2        2016-03-26
3        2016-03-12
4        2016-04-01
            ...    
49995    2016-03-27
49996    2016-03-28
49997    2016-04-02
49998    2016-03-08
49999    2016-03-13
Name: ad_created, Length: 50000, dtype: object


In [23]:
# The code below outputs the distribution of values in the 'date_crawled' column
# It also outputs the relative frequency of the values in the 'date_crawled' column'

autos['date_crawled'].str[:10].value_counts(normalize = True, dropna = False).sort_index()

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

In [20]:
# The below code produces the same output as the code above but sorts the output by values in descending order

autos['date_crawled'].str[:10].value_counts(normalize = True, dropna = False).sort_values(ascending = False)

2016-04-03    0.03868
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-12    0.03678
2016-03-14    0.03662
2016-04-04    0.03652
2016-03-07    0.03596
2016-04-02    0.03540
2016-03-19    0.03490
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-15    0.03398
2016-04-01    0.03380
2016-03-30    0.03362
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-22    0.03294
2016-03-11    0.03248
2016-03-26    0.03248
2016-03-23    0.03238
2016-03-10    0.03212
2016-03-31    0.03192
2016-03-25    0.03174
2016-03-17    0.03152
2016-03-27    0.03104
2016-03-16    0.02950
2016-03-24    0.02910
2016-03-05    0.02538
2016-03-13    0.01556
2016-03-06    0.01394
2016-04-05    0.01310
2016-03-18    0.01306
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

##### After exploring the distribution of the values within the 'date_crawled' column further, we observe that the crawler was used daily for about a month (2016-03-05 to 2016-04-07). We also observe the listings crawled on each day is roughly uniform.

##### Lets move on to the next column -- 'last_seen'

In [42]:
autos['last_seen'].str[:10].value_counts(normalize = True, dropna = False).sort_index()

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

##### The values in this column represent the last time the crawler saw any listings which allows us to determine what day a listing may have been removed, presumably because the car was sold. 

##### The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [41]:
autos['ad_created'].str[:10].value_counts(normalize = True, dropna = False).sort_index()

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
               ...   
2016-04-03    0.03892
2016-04-04    0.03688
2016-04-05    0.01184
2016-04-06    0.00326
2016-04-07    0.00128
Name: ad_created, Length: 76, dtype: float64

##### With the 'ad_created' column, we observe a wide variety of ad created dates. Most of these tend to fall within 1-2 months of the listing date but others are a lot older.

In [43]:
autos['registration_year'].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

##### Looking at the output from the 'registration_year' column, which indicates the age of the car, we see some odd things. We notice the min value is 1000 which is long before cars were invented and the max value is 9999 which is long into the future. 

##### Lets look into the 'registration_year' column further.

# Dealing with Incorrect Registration Year Data

##### As I pointed out above,the min and max year values for the registration_year column are odd. Because a car can't be first registered until after the listing was seen, any vehicle with a registration year above 2016 is wrong. Figuring out the earliest valid year is more difficult as it could realistically be as far back as 1900's. 

##### I'm going to count the number of listings with cars that fall outside of 1900-2016 interval and see if removing those rows entirely is safe or not.

In [57]:
(~autos['registration_year'].between(1900, 2016)).sum() / autos.shape[0]

0.03944

##### We can see from the output above that when we remove any rows outside of out 1900-2016 interval, we're removing effectively ~4% of our data which isn't much. As a result, we'll remove these rows.

In [58]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.069834
2005    0.062776
1999    0.062464
2004    0.056988
2003    0.056779
2006    0.056384
2001    0.056280
2002    0.052740
1998    0.051074
2007    0.047972
Name: registration_year, dtype: float64

We can see from the output above that most of the vehicles were first registered in the past 20 years.

# Exploring Price by Brand

##### In this section, I will use aggregation techniques to analyze and get a better understanding of the 'brand' column.

In [68]:
# Lets take a look at the values in our 'brand' column

print(autos['brand'].unique())
print()
print(autos['brand'].value_counts(normalize = True))

['peugeot' 'bmw' 'volkswagen' 'smart' 'ford' 'chrysler' 'seat' 'renault'
 'mercedes_benz' 'audi' 'sonstige_autos' 'opel' 'mazda' 'porsche' 'mini'
 'toyota' 'dacia' 'nissan' 'jeep' 'saab' 'volvo' 'mitsubishi' 'jaguar'
 'fiat' 'skoda' 'subaru' 'kia' 'citroen' 'chevrolet' 'hyundai' 'honda'
 'daewoo' 'suzuki' 'trabant' 'land_rover' 'alfa_romeo' 'lada' 'rover'
 'daihatsu' 'lancia']

volkswagen        0.212126
bmw               0.110019
opel              0.108166
mercedes_benz     0.095361
audi              0.086387
ford              0.069793
renault           0.047347
peugeot           0.029524
fiat              0.025860
seat              0.018177
skoda             0.016032
mazda             0.015137
nissan            0.015095
citroen           0.013929
smart             0.013909
toyota            0.012472
sonstige_autos    0.010952
hyundai           0.009848
volvo             0.009245
mini              0.008641
mitsubishi        0.008141
honda             0.007850
kia               0.00710

##### We can see from our output above that in Germany, there are around 40 different brands of vehicles. Not surprisingly, 4 out of the top 5 car brands are all German manufactures. Volkswagon is by far the most popular car being listed at approximately double the rate of the next two car brands combined. 

##### Because there are a lot of brands without a siginificant percentage of listings, I'll limit the rest of my analysis to brands representing more than 5% of the total listings. 

In [69]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [72]:
brand_mean_prices = {}

for brand in common_brands:
  brand_only = autos[autos['brand'] == brand]
  mean_price = brand_only['price'].mean()
  brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

{'audi': 9093,
 'bmw': 8334,
 'ford': 7263,
 'mercedes_benz': 30317,
 'opel': 5252,
 'volkswagen': 6516}

##### Of the top 5 brands we can see a clear distinction in the price gap:
* Audi, BMW and Mercedes Benz are more expensive
* Ford and Opel are less expensive
* Volkswagen is in between - this may explain its popularity, it may be a 'best of 'both worlds' option.

# Exploring Mileage Data

##### In this last section, I'm going to explore the top 6 brands to better understand the average mileage for those cars and if there's any link with mean price. 

In [73]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns=["mean_price"])

,mean_price
volkswagen,6516
bmw,8334
opel,5252
mercedes_benz,30317
audi,9093
ford,7263


In [75]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [76]:
brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,132434
mercedes_benz,130860
audi,129287
opel,129227
volkswagen,128730
ford,124046


In [79]:
brand_info["mean_price"] = mean_prices
brand_info.sort_values(by = 'mean_price', ascending = False)

,mean_mileage,mean_price
mercedes_benz,130860,30317
audi,129287,9093
bmw,132434,8334
ford,124046,7263
volkswagen,128730,6516
opel,129227,5252


##### We can see from the output above that the distribution of mean mileage for the top 6 brands is pretty uniform. Ford is the brand with the lowest mean mileage while BMW is the brand with the highest mean mileage. While the trend is slight, it does seem the more expensive cars have a higher mean mileage than the less expensive ones. That being said, the link between price and mileage is pretty weak. 